In [5]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re
from urllib.parse import urlparse

In [15]:
# path to csv file
data = pd.read_csv("../startup_list_3501-5000.csv")

In [16]:
data.head()

,Unnamed: 0,name,state,linkedin_industry,website_url,check_robots
0,3501,DeepEyes,Bayern,Industrial & Basic Materials,http://deepeyes.co/,True
1,3502,Instafreight,Berlin,"Automotive, Logistics & Mobility",https://www.instafreight.de:443/,False
2,3503,smartUP,Baden-Württemberg,Construction/Real Estate,http://www.smartup-immobilien.de,True
3,3504,getnow,Bayern,Nutrition & Consumables,https://www.getnow.com/,False
4,3505,BeeSonix,Berlin,IT and Communications,http://beesonix.de/,False


In [17]:
def is_scraping_allowed(url):
    """
    Checks if scraping is allowed for the given URL.

    Args:
        url (str): The URL to check.

    Returns:
        bool: True if scraping is allowed, False otherwise.
    """
    # Parse the URL to extract the domain.
    domain = urlparse(url).netloc

    # Get the URL for the robots.txt file for the domain.
    robots_url = f"https://{domain}/robots.txt"

    # Send a GET request to the robots.txt file.
    try:
        response = requests.get(robots_url)
    except:
        # If there was an error, assume scraping is not allowed.
        return False

    # Check if the status code of the response is OK.
    if response.status_code != 200:
        # If not, assume scraping is not allowed.
        return False

    # Parse the robots.txt file.
    robots_txt = response.text

    # Check if the user-agent "*" is disallowed from scraping any pages.
    #if "User-agent: *\nDisallow: /" in robots_txt:
    #    return False

    # Check if the user-agent "*" is specifically disallowed from scraping the given URL.
    if f"User-agent: *\nDisallow: {url}" in robots_txt:
        return False

    # If none of the above conditions are met, assume scraping is allowed.
    return True

In [18]:
def getWebsiteText(url):
    # Headers with Accept-Language set to English
    headers = {
        'Accept-Language': 'en-US,en;q=0.1',
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'
    }

    # Send a request to the website and get the response
    try:
        response = requests.get(url, headers=headers)
        if response.status_code == 200:
            #print(f"The link {url} is working.")
            # Parse the HTML content with BeautifulSoup
            soup = BeautifulSoup(response.content, 'html.parser')
            text = soup.getText()
            return text
        else:
            return f"Error: {response.status_code}"

    except Exception as e:
        # Handle the error here
        if 'response' not in locals():
            return "Error: Failed to get response"
        return f"Error: {response.status_code}"

In [51]:
from bs4 import BeautifulSoup
import requests

def remove_cookie_banners(soup):
    # Find and remove elements with classes containing classes_to_remove content
    classes_to_remove = ['cookie', 'banner', 'popup', 'modal', "cc-dialog", "gdpr-content", "ccform"]  # Customize as needed
    for class_name in classes_to_remove:
        elements = soup.select(f'[class*={class_name}]')
        for element in elements:
            element.extract()

    # Find and remove elements with IDs containing cookie-law-info-bar
    elements = soup.select('[id*=cookie-law-info-bar]')
    for element in elements:
        element.extract()

    # Find and remove elements with classes or IDs containing classes_to_remove content
    pattern = re.compile(r'class=["\'].*?\bcookie\b.*?["\']|id=["\'].*?\bcookie\b.*?["\']')
    tags_with_cookie = soup.find_all(attrs={"class": pattern, "id": pattern})
    for tag in tags_with_cookie:
        tag.extract()
    # Find and remove elements with classes or IDs containing classes_to_remove content
    pattern = re.compile(r'class=["\'].*?\bcc\b.*?["\']|id=["\'].*?\bcc\b.*?["\']')
    tags_with_cookie = soup.find_all(attrs={"class": pattern, "id": pattern})
    for tag in tags_with_cookie:
        tag.extract()

def getWebsiteText(url):
    # Headers with Accept-Language set to English and Cookie consent
    headers = {
        'Accept-Language': 'en-US,en;q=0.1',
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3',
        'Cookie': 'consent=true'  # Add the cookie consent header
    }

    # Send a request to the website and get the response
    try:
        response = requests.get(url, headers=headers)
        if response.status_code == 200:
            # Parse the HTML content with BeautifulSoup
            soup = BeautifulSoup(response.content, 'html.parser')
            remove_cookie_banners(soup)  # Remove elements with classes or IDs containing specified content
            text = soup.get_text()
            return text
        else:
            return f"Error: {response.status_code}"

    except Exception as e:
        # Handle the error here
        if 'response' not in locals():
            return "Error: Failed to get response"
        return f"Error: {response.status_code}"


Check the robots.txt of each web page, by adjusting range(1751, 3500) 
-> all data: len(data)

In [40]:
for i in range(1751, 3500):
  print(i) # only progress information
  url = data['website_url'][i]
  result = is_scraping_allowed(url)
  data.loc[i, "check_robots"] = result

1751


KeyError: 1751

In [ ]:
#Save data as new csv
data.to_csv("/content/drive/MyDrive/")

In [20]:
data.head()

,Unnamed: 0,name,state,linkedin_industry,website_url,check_robots
0,3501,DeepEyes,Bayern,Industrial & Basic Materials,http://deepeyes.co/,True
1,3502,Instafreight,Berlin,"Automotive, Logistics & Mobility",https://www.instafreight.de:443/,False
2,3503,smartUP,Baden-Württemberg,Construction/Real Estate,http://www.smartup-immobilien.de,True
3,3504,getnow,Bayern,Nutrition & Consumables,https://www.getnow.com/,False
4,3505,BeeSonix,Berlin,IT and Communications,http://beesonix.de/,False


In [21]:
#Counts True and False values in column "check_robots"
data["check_robots"].value_counts()

check_robots
True     990
False    509
Name: count, dtype: int64

Gets Text from the allowed websites and saves it as txt

In [41]:
#path for the text files 
import os
path = "./txt_files_3501_5000/"
os.makedirs(path, exist_ok=True)

In [ ]:
for i in range(0, 1501):
  if(data["check_robots"][i]==True):
    url = data['website_url'][i]
    text = getWebsiteText(url)
    lines = text.splitlines()
    new_lines = [line for line in lines if line]
    new_text = "\n".join(new_lines)
    # lowercase + remove everything in name but chars
    # because some names contains symbols e.g. ":" -> would destroy a textfile
    name = data["name"][i].lower()
    name = re.sub(r"[^a-z]", "", name)
    print(f"{i}: {name}") # only progress information
    with open(path+name+".txt", "w", encoding="utf-8") as f:
      f.write(new_text)
    

0: deepeyes
2: smartup
5: surgmark
6: rivr
7: soundnotation
9: fixario
10: wix
12: zvarmcom
14: tibeo
15: beemgee
17: wechselgott
20: creacando
21: animatch
22: oskarde
23: legalhero
24: clecta
25: maschinenfrholz
26: statpile
27: eless
31: optonlinephysiotherapie
32: perun
34: dynamicbiosensors
35: artmano
36: creativequantum
37: erento
38: commerceconnector
39: kaduda
40: axelspringerhytechnologies
42: timebro
44: yoochoose
45: cordition
48: hometogo
49: heavenhr
50: virtualretail
51: promipool
52: reparaturmacherdasvergleichsportalfrreparaturen
54: virtonomy
57: roestbar
59: exocad
62: brainscent
64: studentengo
66: novihumtechnologies
67: quadratologo
68: kontist
70: audibene
71: buynomics
72: yorxs
73: cbddeal
74: b
76: meintierbestatterde
77: rententippsde
78: telelook
79: nuicosmetics
80: instrumentsofthings
81: mambu
82: applyojena
83: y
84: betriebsmittelhelden
85: kinderheldin
86: kartenmacherei
87: innokrobotics
88: deutscheautohaus
89: artflakes
